In [1]:
%cd speech2text_data

/data/group1/z44383r/dev/rl-nlg/experiments/speech2text_data


In [2]:
import os
import wave
import pandas as pd
from tqdm import tqdm
from common_utils.path import ROOT_DPATH
from common_utils.multiwoz_data import MultiWOZData

In [4]:
def audio_stats(audio_fpath):
    with wave.open(audio_fpath,  'rb') as wr:
        ch = wr.getnchannels()
        width = wr.getsampwidth()
        fr = wr.getframerate()
        fn = wr.getnframes()

    stats = {
        "channedls": ch,
        "sample_width": width,
        "sample_rate": fr,
        "frame_rate": fn,
        "length": 1.0*fn/fr
    }
    return stats

def main(part, side, multiwoz_data):
    assert side in ["sys", "user"]
    audio_dpath = os.path.join(ROOT_DPATH, "experiments/text2speech_data/outputs/multiwoz", part)
    pbar = tqdm(multiwoz_data[part])
    all_stats = []
    for dial_name in pbar:
        pbar.set_description(f"Processing {dial_name}")
        audio_dial_dpath = os.path.join(audio_dpath, dial_name)
        os.makedirs(audio_dial_dpath, exist_ok=True)
        for i, s, turn in multiwoz_data.iter_dialog_log(part=part, dial_name=dial_name):
            audio_fpath = os.path.join(audio_dial_dpath, f"{i}.wav")
            if s != side or not os.path.isfile(audio_fpath):
                continue
            all_stats.append(audio_stats(audio_fpath))
    return pd.DataFrame(all_stats)

In [6]:
multiwoz_data = MultiWOZData(rm_ws_before_punc=True)
df = main("train", "sys", multiwoz_data)

2022-04-30 11:02:59,703 : WARNING - multiwoz_data.py - After removing white spaces before punctuation, span_info may become inconsistent.
Processing MUL0501: 100%|██████████| 8434/8434 [16:47<00:00,  8.37it/s] 


In [8]:
df.mean()

channedls           1.000000
sample_width        2.000000
sample_rate     16000.000000
frame_rate      90598.853692
length              5.662428
dtype: float64